In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re

In [2]:
newdata=pd.read_csv("../input/oathss/oaths.csv")

In [3]:
newdata["transcription_text"][0]

'[M177 ENCL]\r\n[[pre-printed]] I, [[handwritten]] I C Meekins [[/handwritten]], of [[handwritten]] Currituck [[/handwritten]] County, State of North Carolina, do solemnly swear or affirm, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves.  So help me God. \r\n\r\n[[handwritten]] Isaac C. Meekins [[/handwritten]]\r\n\r\nSworn and subscribed to this the [[handwritten]] 16 [[/handwritten]] day of [[handwritten]] Sept [[/handwritten]] A.D., 1865, before \r\n[[handwritten]] E H Walker [[/handwritten]], J.P.\r\nIt is hereby certified that the above is a true copy of the original oath taken and subscribed by [[handwritten]] I C Meekins\r\nE H Walker, J.P.\r\nJohn B Etheridge, J.P.\r\n[[\\ha

In [4]:
newdata["transcription_text"][9]

' I, T A Dough, of Currituck County, State of North-Carolina, do solemnly swear or affirm, in presence of Almighty God, that I will henceforth faithfully support, protect and defend the Constitution of the United States and the Union of the States thereunder; and that I will, in like manner, abide by and faithfully support all laws and proclamations which have been made during the existing rebellion with reference to the emancipation of slaves. So help me God. \r\nThos A Dough\r\n\r\nSworn and subscribed to this the 16 day of Sept, \r\nA.D., 1865, before \r\nJohn B Etheridge, J. P.\r\n\r\nIt is hereby certified that the above is a true copy of the original oath taken and subscribed by T A Dough\r\nJohn B Etheridge, J. P.\r\nE H Walker, J. P.\r\n[[blank line]], J. P.\r\n'

In [5]:
newdata["transcription_text"][100]

'[[preprinted]]\r\nState of North Carolina, }\r\n^[[Randolph]] County. }\r\n\r\nI, ^[[W R Frazier]], do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought nor accepted, nor attempted to exercise the functions of any office whatever under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power or constitution within the United States, hostile or inimical thereto. And I do further swear, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any mental reservation or purp

In [6]:
newdata["transcription_text"][1500]

'I J A Sanders solemnly Swear that I have carefully read the Amnesty Proclamation issued by ANDREW JOHNSON, President of the United States of America, on May 29th, 1865, and that I am not excepted from the benefits of that Proclamation by any one of the fourteen exceptions therein made, [[strikethrough]] except [[/strikethrough]]\r\nLaura Sanders\r\n\r\nSworn to and subscribed before me at Charleston SC this 7th day of Oct 1865.\r\nJ Milton Thompson\r\nCapt 33 U.S.C.T. & Pro. Marshal\r\n\r\n[[left margin]] Copy  J Milton Thompson capt 33 USCT & Pro Marshal City of Charleston [[/left margin]]\r\n\r\nNo. 1619\r\nUnited States of America.\r\n\r\nI J A L. Sanders of the County of Charleston State of South Ca do solemnly Swear in the presence of Almighty God, that I will henceforth faithfully support and defend the Constitution of the United States, and the Union of the States thereunder, and that I will, in like manner, abide by and faithfully support all laws and proclamations which have 

In [7]:
newdata["transcription_text"][1980]

'United States OF AMERICA.\r\n[[image]] stamped eagle [[/image]]\r\nTEST OATH. \r\n\r\n\r\nI, J. B. Healy - do solemnly swear that I have never voluntarily borne arms against the United States since I have been a citizen thereof; that I have voluntarily given no aid, countenance, counsel, or encouragement to persons engaged in armed hostility thereto; that I have neither sought, nor accepted, nor attempted to exercise the functions of any office whatever, under any authority, or pretended authority, in hostility to the United States; that I have not yielded a voluntary support to any pretended government, authority, power, or constitution within the United States, hostile or inimical thereto.\r\nAnd I do further swear, or affirm, that, to the best of my knowledge and ability, I will support and defend the Constitution of the United States against all enemies, foreign and domestic; that I will bear true faith and allegiance to the same; that I take this obligation freely, without any me

In [8]:
newdata["transcription_text"][2100]

'Oath\r\nArington Dilworth\r\nGuilford Co No Ca\r\n\r\nAug 14, 66'

# Preprocessing

In [9]:
all_text=newdata["transcription_text"]

In [10]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [11]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# delete just stop words

In [12]:
punct='!"#$%&\'()*+-/<=>?@[\\]^`{|}~'

In [13]:
def process(s):
    s= re.sub("([\(\[]).*?([\)\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [14]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = process(all_text[i])

100%|██████████| 2136/2136 [00:08<00:00, 250.87it/s]


In [15]:
for i in tqdm(range(newdata.shape[0])):
    all_text[i] = " ".join(all_text[i])

100%|██████████| 2136/2136 [00:00<00:00, 2929.40it/s]


In [16]:
all_text

0       I , I C Meekins , of Currituck County , State ...
1       Department of Virginia North Carolina , 1864. ...
2       I , W. H. Pearce , of Craven County , State of...
3       State of North Carolina , ________ County . I ...
4       State of North Carolina , Wake County . I , S....
                              ...                        
2131    Office Provost Marshal Raleigh , N.C. May 26 6...
2132    July 30th 1867 Birnie Wm Agent Bu R F A L Oath...
2133    State of North Carolina , Robeson County . I ,...
2134    NARA 449 102 We whose name written well person...
2135    NARA 450 404 Office Provost Marshall City of W...
Name: transcription_text, Length: 2136, dtype: object

# Flair

In [17]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-p

# Hugging face  : Flair

**ner-english** 

In [18]:
!git lfs install
!git clone https://huggingface.co/flair/ner-english-large

git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'ner-english-large'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 11.03 KiB | 1.38 MiB/s, done.


In [19]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-11 01:27:42,370 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-11 01:28:07,908 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [20]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-11 01:29:48,205 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-11 01:30:05,242 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


tagger3 nerenglishlarge

In [21]:
listenglarge=[]
label=[]
Per=[]
ORG=[]
LOC=[]
from flair.data import Sentence

dflarge=pd.DataFrame({"text":"", "PERSON-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        dflarge.loc[i,"text"]=all_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listenglarge.append(entity)
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") | (dc[j]=="MISC") :
                      O.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
            

dflarge["PERSON-nerenglarge"]=Per
dflarge["LOC-nerenglarge"]=LOC
dflarge["ORG-nerenglarge"]=ORG                        

In [22]:
dflarge

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,"I , I C Meekins , of Currituck County , State ...","[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,"Department of Virginia North Carolina , 1864. ...",[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,"I , W. H. Pearce , of Craven County , State of...","[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,"State of North Carolina , ________ County . I ...",[God],"[State of, North Carolina, United States]",[Constitution]
4,"State of North Carolina , Wake County . I , S....","[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"
...,...,...,...,...
2131,"Office Provost Marshal Raleigh , N.C. May 26 6...","[Geo . Little, Geo, B. Ayer, Hugo Hillebrandt]","[Raleigh, N.C., Wake County, State of N.C, Uni...",[]
2132,July 30th 1867 Birnie Wm Agent Bu R F A L Oath...,[],[],[]
2133,"State of North Carolina , Robeson County . I ,...","[William Birnie, God, Wm Birnie, W.A]","[State of, North Carolina, Robeson County, Uni...",[Constitution]
2134,NARA 449 102 We whose name written well person...,"[Price, James Fulton, Sgd W . N. Peden, Miles ...","[State of North Carolina, New Hanover County, ...","[Fulton Price, Confederate Government, Court o..."


tagger 2 ontonotes

In [23]:
listoto=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
from flair.data import Sentence

aldf=pd.DataFrame({"text":"", "PERSON-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(2136):
        sentencess =Sentence(all_text[i])
        aldf.loc[i,"text"]=all_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        O=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
                        listoto.append(entity)
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
            
aldf["Date-ontonotes"]=Date
aldf["PERSON-ontonotes"]=Per
aldf["GPE-ontonotes"]=GPE
aldf["ORG-ontonotes"]=ORG    

In [24]:
aldf.shape

(2136, 5)

In [25]:
aldf.head()

,text,PERSON-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes
0,"I , I C Meekins , of Currituck County , State ...","[I C Meekins, Isaac C. Meekins, E H Walker, Jo...","[Currituck County ,]",[],"[16 day of Sept A.D . , 1865]"
1,"Department of Virginia North Carolina , 1864. ...",[],"[North Carolina, United States, States]","[Department of Virginia, State]",[1864]
2,"I , W. H. Pearce , of Craven County , State of...","[W. H. Pearce, W . H. Pearce]",[],[],"[3rd day of October , A.D. , 1865]"
3,"State of North Carolina , ________ County . I ...",[],"[North Carolina, United States]",[],"[______ day of ____________ , 1866]"
4,"State of North Carolina , Wake County . I , S....",[S.E . Allen],"[North Carolina, Wake County, United States]",[],"[31st day of May , 1867]"


In [26]:
aldf.shape

(2136, 5)

In [27]:

LDate=[]
for i in aldf["Date-ontonotes"]:
    for j in i:
        LDate.append(j)
len(LDate)

4887

**5000 dates**

# Post processing

**For dates**

In [28]:
G=[]
for i in aldf["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-ZZa-z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)

    
aldf["postDate-ontonotes"]=GF
            
            
        

# Run this to see the extracted dates before filtring them 

In [29]:
#GF

In [30]:
G=[]
for i in aldf["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

aldf["postDate-ontonotes"]=G

**great !**

****

****

In [31]:
dflarge.head()

,text,PERSON-nerenglarge,LOC-nerenglarge,ORG-nerenglarge
0,"I , I C Meekins , of Currituck County , State ...","[I C Meekins, Almighty, God, Isaac C. Meekins,...","[Currituck County, State of North Carolina, Un...",[Constitution]
1,"Department of Virginia North Carolina , 1864. ...",[],"[North Carolina, United States, States]","[Department of Virginia, OATH PAROLE]"
2,"I , W. H. Pearce , of Craven County , State of...","[W. H. Pearce, Almighty, God, E. Ampetts, W . ...","[Craven County, State of NorthCarolina, United...",[Constitution]
3,"State of North Carolina , ________ County . I ...",[God],"[State of, North Carolina, United States]",[Constitution]
4,"State of North Carolina , Wake County . I , S....","[S.E, God, T.D, McAlpine]","[State of, North Carolina, Wake County, United...","[Constitution, Res . Corp]"


In [32]:
aldf["Person-NerEngLarge"]=dflarge["PERSON-nerenglarge"]
aldf["LOC-NerEngLarge"]=dflarge["LOC-nerenglarge"]
aldf["ORG-NerEngLarge"]=dflarge["ORG-nerenglarge"]


In [33]:
allPerson=[]
for i in aldf["PERSON-ontonotes"]:
            allPerson.append(i)
allDatess=[]
for i in aldf["Date-ontonotes"]:
            allDatess.append(i)
allpostdates=[]
for i in aldf["postDate-ontonotes"]:
            allpostdates.append(i)
allGPE=[]
for i in aldf["GPE-ontonotes"]:
            allGPE.append(i)
alORGl=[]
for i in aldf["ORG-ontonotes"]:
            alORGl.append(i)

****

In [34]:
Personlarge=[]
for i in dflarge["PERSON-nerenglarge"]:
            Personlarge.append(i)
ORGlarge=[]
for i in dflarge["ORG-nerenglarge"]:
            ORGlarge.append(i) 
LOClarge=[]
for i in dflarge["LOC-nerenglarge"]:
            LOClarge.append(i)

* as usual , ner English Large is better in extracting Person 
* ontonotes is thes best in extracting dates after good postprocessing and filtring almost all pattern 
* ner Engish Large  also better in Location and Oranization 

# Person Large (without punct)

In [35]:
all_text[0]

'I , I C Meekins , of Currituck County , State of North Carolina , solemnly swear affirm , presence of Almighty God , I henceforth faithfully support , protect defend Constitution of United States Union of States thereunder ; I , like manner , abide faithfully support law proclamation made existing rebellion reference emancipation of slave . So help God . Isaac C. Meekins Sworn subscribed 16 day of Sept A.D. , 1865 , E H Walker , J.P . It hereby certified true copy of original oath taken subscribed I C Meekins E H Walker , J.P. John B Etheridge , J.P .'

In [36]:
PL=[]
for i in Personlarge:
    for j in i:
        PL.append(j)

In [37]:
len(PL)

9512

# Person ontonotes (without punct)

In [38]:
PAu=[]
for i in allPerson:
    for j in i:
        PAu.append(j)

In [39]:
len(PAu)

5393

****

****

# dates

**run those cells to take a look at the result more clearly**

In [40]:
#G

In [41]:
# import pickle
# my_pickled_list = pickle.dumps(listoto)  # Pickling the object
# print(f"This is my pickled object:\n{my_pickled_list}\n")

# my_pickled_list

# my_unpickled_object = pickle.loads(my_pickled_list)  # Unpickling the object
# print(
#     f"This is a_dict of the unpickled object:\n{my_unpickled_object}\n")


# my_unpickled_object

# standardizing dates without punct

In [42]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY","Feb"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

# Final date 

In [43]:
#NG

In [44]:
Lg=[]
for i in G:
    for j in i:
        Lg.append(j)

In [45]:
len(Lg)

3648

In [46]:
Lng=[]
for i in NG:
    for j in i:
        Lng.append(j)

In [47]:
len(Lng)

3587

In [48]:
NG

[['16-Septembre-1865'],
 [],
 ['3-October-1865'],
 [],
 ['31-May-1867'],
 ['1-Dec.-1866'],
 ['30-May-1864'],
 ['16-Septembre-1865'],
 ['16-Nov.-1866'],
 ['16-Septembre-1865'],
 ['4-November-1867'],
 ['4-November-1867'],
 ['2-October-1865'],
 [],
 ['31-December-1867'],
 ['19-June-1865', '19-June-1865'],
 ['4-November-1865'],
 ['9-November-1866',
  '9-November-1866',
  '9-November-1866',
  'The 9th -November-1866'],
 ['1-December-1866', '1-Dec.-1866'],
 ['25-June-1864'],
 ['20-October-1866'],
 [],
 ['20-July-1867', '24-July-1867'],
 ['19-May-1865', '19-May-1865', '19-May-1865'],
 [],
 ['20-October-1866'],
 ['15-January-1868'],
 ['1-July-1867'],
 [],
 [],
 [],
 [],
 ['3-August-1867'],
 ['15-December-67'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['23-July-1867'],
 ['26-July-1867'],
 ['25-July-1867'],
 [],
 [],
 ['22-August-66'],
 [],
 ['24-August-66'],
 [],
 ['29-August-66'],
 [],
 ['1-November-1866'],
 [],
 ['9-November-66'],
 [],
 [],
 ['26-July-1867'],
 ['31-July-1868'],
 [],
 ['18-January-1868'